# One- and Two-Qubit Randomized Benchmarking in LabOne Q with Qiskit

In this notebook, we'll use the [Qiskit Experiment Library](https://qiskit.org/ecosystem/experiments/apidocs/library.html) to generate one and two qubit randomized benchmarking experiments. 
We'll then export the generated experiment to [OpenQASM](https://openqasm.com/), import these OpenQASM experiments into LabOne Q, compile, and simulate the output signals.

When generating randomized benchmarking experiments in Qiskit, it will return a list of quantum circuits with the specified parameters. 
We show here how to efficiently import, compile and execute such a list into LabOne Q, resulting in a single, large experiment.

## Imports

In [ ]:
from __future__ import annotations

# LabOne Q:
# additional imports
# device setup and descriptor
from laboneq import openqasm3
from laboneq.contrib.example_helpers.generate_device_setup import (
    generate_device_setup_qubits,
)

# plotting functionality
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

# core LabOne Q functionality
from laboneq.simple import *

# qiskit
from qiskit import qasm3, transpile
from qiskit_experiments.library import randomized_benchmarking

## Define the Experimental Setup

Below, we generate a pre-calibrated experimental setup containing a [DeviceSetup](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/00_set_up_equipment.html#devicesetup), [Transmon qubits](https://docs.zhinst.com/labone_q_user_manual/core/reference/dsl/quantum.html?h=transmon#laboneq.dsl.quantum.transmon.Transmon) and a [Session](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/01_session/concepts/00_session.html?h=session) to run the experiments. 

In [ ]:
# specify the number of qubits you want to use
number_of_qubits = 2

# generate the device setup and the qubit objects using a helper function
device_setup, qubits = generate_device_setup_qubits(
    number_qubits=number_of_qubits,
    pqsc=[{"serial": "DEV10001"}],
    hdawg=[{"serial": "DEV8001", "zsync": 0, "number_of_channels": 8, "options": None}],
    shfqc=[
        {
            "serial": "DEV12001",
            "zsync": 1,
            "number_of_channels": 6,
            "readout_multiplex": 6,
            "options": None,
        }
    ],
    include_flux_lines=True,
    server_host="localhost",
    setup_name=f"my_{number_of_qubits}_tuneable_qubit_setup",
)

q0, q1 = qubits[:2]

In [ ]:
# create and connect to Session

# use emulation mode - no connection to instruments
use_emulation = True

my_session = Session(device_setup=device_setup)
my_session.connect(do_emulation=use_emulation, reset_devices=True)

## Defining the QPU and Quantum Operations

Here, we define the QPU and the class of quantum operations corresponding to the gates in our QASM programs for Randomized Benchmarking produced with Qiskit. 

To learn more about quantum operations, check out these pages in the LabOne Q User's Manual, [here](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/03_sections_pulses_and_quantum_operations/concepts/08_quantum_operations.html) and [here](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/quantum_operations.html). To learn more about the QPU, have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/getting_started.html).

In [ ]:
class QASMOperations(dsl.QuantumOperations):
    """Class implementing the collection of quantum operations.

    Operations for the QASM RB experiments created by Qiskit.
    """

    QUBIT_TYPES = Transmon

    @dsl.quantum_operation
    def delay(self, q: Transmon, time: float) -> None:
        """A delay operation on the drive signal of the qubit."""
        dsl.delay(q.signals["drive"], time=time)

    @dsl.quantum_operation
    def x(
        self,
        q: Transmon,
        amplitude_scale: float = 1.0,
        amplitude: float | None = None,
        label: str = "x",
    ) -> None:
        """A drag pulse implementing an x rotation.

        The pulse length and amplitude are taken from the qubit parameters.
        """
        pulse_parameters = {"function": "drag"}
        x_pulse = dsl.create_pulse(pulse_parameters, name=f"{q.uid}_{label}")
        if amplitude is None:
            amplitude = amplitude_scale * q.parameters.user_defined["amplitude_pi"]
        dsl.play(
            q.signals["drive"],
            amplitude=amplitude,
            length=q.parameters.user_defined["pulse_length"],
            pulse=x_pulse,
        )

    @dsl.quantum_operation
    def sx(self, q: Transmon) -> None:
        """An sx operation used in the RB decomposition.

        Calls the x operation with a fixed amplitude_scale of 0.5.
        """
        self.x.omit_section(q, amplitude_scale=0.5, label="sx")

    @dsl.quantum_operation
    def rz(self, q: Transmon, angle: float) -> None:
        """An operation implementing a z rotation by the given angle."""
        dsl.play(
            signal=q.signals["drive"],
            pulse=None,
            increment_oscillator_phase=angle,
        )

    @dsl.quantum_operation
    def measure(self, q: Transmon, handle: str) -> None:
        """An operation implementing a qubit measurement.

        The results are stored under the name given by handle. The readout
        and integration parameters are taken from the qubit.
        """
        ro_pulse_parameters = {"function": "gaussian_square", "zero_boundaries": True}
        ro_pulse = dsl.create_pulse(ro_pulse_parameters, name=f"{q.uid}_readout_pulse")
        int_pulse_parameters = {"function": "const"}
        kernels = [
            dsl.create_pulse(int_pulse_parameters, name=f"{q.uid}_integration_kernel")
        ]
        dsl.measure(
            measure_signal=q.signals["measure"],
            measure_pulse_amplitude=q.parameters.user_defined["readout_amplitude"],
            measure_pulse_length=q.parameters.user_defined["readout_length"],
            measure_pulse=ro_pulse,
            handle=handle,
            acquire_signal=q.signals["acquire"],
            integration_kernel=kernels,
            integration_length=q.parameters.user_defined["readout_length"],
            reset_delay=None,
        )

    @dsl.quantum_operation
    def reset(self, q: Transmon) -> None:
        """An operation implementing active reset on a qubit."""
        handle = f"{q.uid}_qubit_state"
        self.measure(q, handle=handle)
        self.delay(q, q.parameters.user_defined["reset_delay_length"])
        with dsl.match(name=f"match_{q.uid}", handle=handle):
            with dsl.case(name=f"case_{q.uid}_g", state=0):
                pass
            with dsl.case(name=f"case_{q.uid}_e", state=1):
                self.x.omit_section(q)

    @dsl.quantum_operation
    def cx(self, q_control: Transmon, q_target: Transmon) -> None:
        """An operation implementing a cx gate on two qubits.

        The controlled X gate is implemented using a cross-resonance gate.
        """
        cx_id = f"cx_{q_control.uid}_{q_target.uid}"

        # define cancellation pulses for target and control
        cancellation_control_n = dsl.create_pulse(
            {"function": "gaussian_square"}, name="CR-"
        )
        cancellation_control_p = dsl.create_pulse(
            {"function": "gaussian_square"}, name="CR+"
        )
        cancellation_target_p = dsl.create_pulse(
            {"function": "gaussian_square"}, name="q1+"
        )
        cancellation_target_n = dsl.create_pulse(
            {"function": "gaussian_square"}, name="q1-"
        )

        # play X pulses on both target and control
        with dsl.section(name=f"{cx_id}_x_both") as x180_both:
            self.x(q_control, label="x180")
            self.x(q_target, label="x180")

        # First cross-resonance component
        with dsl.section(
            name=f"{cx_id}_canc_p", play_after=x180_both.uid
        ) as cancellation_p:
            dsl.play(signal=q_target.signals["drive"], pulse=cancellation_target_p)
            dsl.play(signal=q_control.signals["flux"], pulse=cancellation_control_n)

        # play X pulse on control
        x180_control = self.x(q_control, label="x180")
        x180_control.play_after = cancellation_p.uid

        # Second cross-resonance component
        with dsl.section(
            name=f"cx_{cx_id}_canc_n", play_after=x180_control.uid
        ):
            dsl.play(signal=q_target.signals["drive"], pulse=cancellation_target_n)
            dsl.play(signal=q_control.signals["flux"], pulse=cancellation_control_p)

In [ ]:
from laboneq.dsl.quantum import QPU

qpu = QPU(qubits=[q0, q1], quantum_operations=QASMOperations())

## Single-Qubit Randomised Benchmarking using Qiskit

You'll start by creating Standard RB experiments from the Qiskit Experiment Library [here](https://qiskit.org/ecosystem/experiments/stubs/qiskit_experiments.library.randomized_benchmarking.StandardRB.html#qiskit_experiments.library.randomized_benchmarking.StandardRB). 
Here, we do this for one qubit for a few different RB sequence lengths.

Note that most circuits that can be generated in Qiskit and converted to OpenQASM could be adapted to be run in a similar way in LabOne Q! 

### Define Circuits with Qiskit

In [ ]:
# Use Qiskit Experiment Library to Generate RB
rb1_qiskit_circuits = randomized_benchmarking.StandardRB(
    physical_qubits=[0],
    lengths=[4, 8, 16],
    num_samples=2,
).circuits()

When efficiently importing and executing a list of quantum circuits, there currently are strong limitations as to how the measurements are scheduled in these experiment. 
We strip them here from the Qiskit circuit. We will re-add them to the LabOne Q experiment separately when doing the import.  

In [ ]:
for circuit in rb1_qiskit_circuits:
    circuit.remove_final_measurements()

In [ ]:
rb1_qiskit_circuits[2].draw()

You can then use the Qiskit `transpile` function to obtain a representation of the circuits in your favorite set of basis gates.

Below, we choose the basis `["id", "sx", "x", "rz", "cx"]`. Note that all these gates (except the identity "id") must exist in your set of quantum operations!

In [ ]:
# Choose basis gates
rb1_transpiled_circuits = transpile(
    rb1_qiskit_circuits, basis_gates=["id", "sx", "x", "rz", "cx"]
)

In [ ]:
rb1_program_list = []
for circuit in rb1_transpiled_circuits:
    rb1_program_list.append(qasm3.dumps(circuit))

print(rb1_program_list[2])

### Execute a single QASM program

Now, you'll transpile a single OpenQASM program into a LabOne Q `Experiment` pulse sequence using the class `OpenQASMTranspiler` and the options class `SingleProgramOptions`. Check out the [LabOne Q QASM transpiler tutorial](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/08_openqasm/00_program_to_experiment.html) to learn more about this interface.

Once you've done that, you can compile your `Experiment` and plot the output using the LabOne Q simulator.

**Note**: the parameter `qubit_map` below may need to be updated to match the names of the qubit register from your QASM circuit!

Below, we choose the QASM program defined in the third entry of `rb1_program_list`.

In [ ]:
# Instantiate OpenQASMTranspiler from the QPU
transpiler = openqasm3.OpenQASMTranspiler(qpu)

# Define options
options = openqasm3.SingleProgramOptions()
# We will not change any of the default options

# Create the Experiment
rb1_exp_single_program = transpiler.experiment(
    program=rb1_program_list[2],
    qubit_map={"q": [q0]},
    options=options,
)

# Set the Experiment calibration from the qubit calibration
rb1_exp_single_program.set_calibration(q0.calibration())

# Compile the Experiment
rb1_compiled_exp_single_program = my_session.compile(rb1_exp_single_program)

# Run the Experiment
rb1_results_single_program = my_session.run(rb1_compiled_exp_single_program)

#### Look at the simulated output

In [ ]:
plot_simulation(
    rb1_compiled_exp_single_program,
    length=1.6e-6,
    plot_width=12,
    plot_height=3,
    signal_names_to_show=["drive"],
)

#### Draw the circuit from above

In [ ]:
rb1_transpiled_circuits[2].draw()

#### Look at the pulse sheet

In [ ]:
show_pulse_sheet(name="1-qubit RB", compiled_experiment=rb1_compiled_exp_single_program)

### Execute the full RB Experiment

Below, we will use the `.batch_experiment()` method to create and `Experiment` from our list of QASM programs `rb1_program_list`, containing the full single-qubit RB experiment. 

The entries in `rb1_program_list` are individual RB sequences of a given number of gates, `m`. In total, there will be $m\times K$ sequences in the list, where `K` is the number of randomizations of each length (the `num_samples` parameter in the Qiskit interface above). In our choice above, we have $K=2$ and $m\in \{4, 8, 16\}$.

In the `MultiProgramOptions`, you can use the field `batch_execution_mode` to specify how all these RB sequences in the list should be executed: 

* all in real-time ("rt");
* every sequence of `m` gates in real-time and the iteration over the sequences in near-time ("nt");
* split the entries into the number of near-time steps (called "chunks") using the pipeliner ("pipeline"). Specify the number of chunks to use in the options field `pipeline_chunk_count`.

Below, we use the "pipeline" option, and split our 10 sequences into 2 chunks of 3 RB sequences each. This means that we will have two near-time steps, and each real-time loop will run over 3 RB sequences (all the lengths, in our case).

Note that here we use a different options class, `MultiProgramOptions`.

In [ ]:
# Instantiate OpenQASMTranspiler from the QPU
transpiler = openqasm3.OpenQASMTranspiler(qpu)

# Define options
options = openqasm3.MultiProgramOptions()
options.repetition_time = 20e-5
options.batch_execution_mode = "pipeline"
options.pipeline_chunk_count = 2
options.add_reset = False

# Create the Experiment
rb1_exp = transpiler.batch_experiment(
    programs=rb1_program_list,
    qubit_map={"q": [q0]},
    options=options,
)

# Set the Experiment calibration from the qubit calibration
rb1_exp.set_calibration(q0.calibration())

# Compile the Experiment
rb1_compiled_exp = my_session.compile(rb1_exp)

# Run the Experiment
rb1_results = my_session.run(rb1_compiled_exp)

In [ ]:
## KNOWN ISSUE - pulse sheet viewer and output simulation are not available

### Execute the full RB Experiment - including active qubit reset

Let's re-run the single-qubit RB experiment with active reset. Just set the options field `.add_reset` to `True`.

In [ ]:
# Instantiate OpenQASMTranspiler from the QPU
transpiler = openqasm3.OpenQASMTranspiler(qpu)

# Define options
options = openqasm3.MultiProgramOptions()
options.repetition_time = 20e-5
options.batch_execution_mode = "pipeline"
options.pipeline_chunk_count = 2
options.add_reset = True

# Create the Experiment
rb1_exp_with_reset = transpiler.batch_experiment(
    programs=rb1_program_list,
    qubit_map={"q": [q0]},
    options=options,
)

# Set the Experiment calibration from the qubit calibration
rb1_exp_with_reset.set_calibration(q0.calibration())

# Compile the Experiment
rb1_compiled_exp_with_reset = my_session.compile(rb1_exp_with_reset)

# Run the Experiment
rb1_results_with_reset = my_session.run(rb1_compiled_exp_with_reset)

## Two-Qubit Randomized Benchmarking using Qiskit

### Define Circuits with Qiskit

In [ ]:
# Use Qiskit Experiment Library to Generate RB
rb2_qiskit_circuits = randomized_benchmarking.StandardRB(
    physical_qubits=[0, 1],
    lengths=[4, 8, 16],
    num_samples=2,
).circuits()

When efficiently importing and executing a list of quantum circuits, there currently are strong limitations as to how the measurements are scheduled in these experiment. 
We strip them here from the Qiskit circuit. We will re-add them to the LabOne Q experiment separately when doing the import.  

In [ ]:
for circuit in rb1_qiskit_circuits:
    circuit.remove_final_measurements()

In [ ]:
rb2_qiskit_circuits[0].draw()

You can then use the Qiskit `transpile` function to obtain a representation of the circuits in your favorite set of basis gates. 

Below, we choose the basis `["id", "sx", "x", "rz", "cx"]`. Note that all these gates (except the identity "id") must exist in your set of quantum operations!

In [ ]:
# Choose basis gates
rb2_transpiled_circuits = transpile(
    rb2_qiskit_circuits, basis_gates=["id", "sx", "x", "rz", "cx"]
)

In [ ]:
rb2_program_list = []
for circuit in rb2_transpiled_circuits:
    rb2_program_list.append(qasm3.dumps(circuit))


print(rb2_program_list[0])

### Execute a single QASM program

Now, you'll transpile a two-qubit OpenQASM program into a LabOne Q `Experiment` pulse sequence using the class `OpenQASMTranspiler` and the options class `SingleProgramOptions`. Check out the [LabOne Q QASM transplier tutorial](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/08_openqasm/00_program_to_experiment.html) to learn more about this interface.

Once you've done that, you can compile your `Experiment` and plot the output using the LabOne Q simulator.

**Note**: the parameter `qubit_map` below may need to be updated to match the names of the qubit register from your QASM circuit!

Below, we choose the QASM program defined in the first entry of `rb2_program_list`.

In [ ]:
# Instantiate OpenQASMTranspiler from the QPU
transpiler = openqasm3.OpenQASMTranspiler(qpu)

# Define options
options = openqasm3.SingleProgramOptions()
# We will not change any of the default options

# Create the Experiment
rb2_exp_single_program = transpiler.experiment(
    program=rb2_program_list[0],
    qubit_map={"q": [q0, q1]},
    options=options,
)

# Set the Experiment calibration from the qubit calibrations
rb2_exp_single_program.set_calibration(q0.calibration())
rb2_exp_single_program.set_calibration(q1.calibration())

# Compile the Experiment
rb2_compiled_exp_single_program = my_session.compile(rb2_exp_single_program)

# Run the Experiment
rb2_results_single_program = my_session.run(rb2_compiled_exp_single_program)

#### Look at the simulated output

In [ ]:
plot_simulation(
    rb2_compiled_exp_single_program,
    length=15e-6,
    plot_width=12,
    plot_height=3,
    signal_names_to_show=[
        "q0/drive",
        "q0/flux",
        "q1/drive",
        "q1/flux",
    ],
)

#### Draw the circuit from above

In [ ]:
rb2_transpiled_circuits[0].draw()

#### Look at the pulse sheet

In [ ]:
show_pulse_sheet(
    name="2-qubit RB",
    compiled_experiment=rb2_compiled_exp_single_program,
    max_events_to_publish=10e4
)

### Execute the full RB Experiment

Below, we will use the `.batch_experiment()` method to create and `Experiment` from our list of QASM programs, `rb2_program_list`, containing the full two-qubit RB experiment. 

The entries in `rb2_program_list` are individual RB sequences of a given number of gates, `m`. In total, there will be $m\times K$ sequences in the list, where `K` is the number of randomizations of each length (the `num_samples` parameter in the Qiskit interface above). In our choice above, we have $K=2$ and $m\in \{4, 8, 16\}$.

In the `MultiProgramOptions`, you can use the field `batch_execution_mode` to specify how all these RB sequences in the list should be executed: 

* all in real-time ("rt");
* every sequence of `m` gates in real-time and the iteration over the sequences in near-time ("nt");
* split the entries into the number of near-time steps (called "chunks") using the pipeliner ("pipeline"). Specify the number of chunks to use in the options field `pipeline_chunk_count`.

Below, we use the "pipeline" option, and split our 10 sequences into 2 chunks of 3 RB sequences each. This means that we will have two near-time steps, and each real-time loop will run over 3 RB sequences (all the lengths, in our case).

Note that here we use a different options class, `MultiProgramOptions`.

In [ ]:
# Instantiate OpenQASMTranspiler from the QPU
transpiler = openqasm3.OpenQASMTranspiler(qpu)

# Define options
options = openqasm3.MultiProgramOptions()
options.repetition_time = 100e-5
options.batch_execution_mode = "pipeline"
options.pipeline_chunk_count = 2
options.add_reset = False

# Create the Experiment
rb2_exp = transpiler.batch_experiment(
    programs=rb2_program_list,
    qubit_map={"q": [q0, q1]},
    options=options,
)

# Set the Experiment calibration from the qubit calibrations
rb2_exp.set_calibration(q0.calibration())
rb2_exp.set_calibration(q1.calibration())

# Compile the Experiment
rb2_compiled_exp = my_session.compile(rb2_exp)

# Run the Experiment
rb2_results = my_session.run(rb2_compiled_exp)

In [ ]:
## KNOWN ISSUE - pulse sheet viewer and output simulation are not available

### Execute the full RB Experiment - including active qubit reset

Let's re-run the two-qubit RB experiment with active reset. Just set the options field `.add_reset` to `True`.

In [ ]:
# Instantiate OpenQASMTranspiler from the QPU
transpiler = openqasm3.OpenQASMTranspiler(qpu)

# Define options
options = openqasm3.MultiProgramOptions()
options.repetition_time = 100e-5
options.batch_execution_mode = "pipeline"
options.pipeline_chunk_count = 2
options.add_reset = True

# Create the Experiment
rb2_exp_with_reset = transpiler.batch_experiment(
    programs=rb2_program_list,
    qubit_map={"q": [q0, q1]},
    options=options,
)

# Set the Experiment calibration from the qubit calibrations
rb2_exp_with_reset.set_calibration(q0.calibration())
rb2_exp_with_reset.set_calibration(q1.calibration())

# Compile the Experiment
rb2_compiled_exp_with_reset = my_session.compile(rb2_exp_with_reset)

# Run the Experiment
rb2_results_with_reset = my_session.run(rb2_compiled_exp_with_reset)